**Hybrid**

This paper is the baseline of the RL-QVO.

As discussed in the paper of RL-QVO, usually, subgraph matching algorithm could be divided into three parts. This paper collects some SOTA algorithm in this three field separately and compare the efficiency of them. So actually, this is a survey paper. I'd like to discuss algorithm in this paper in the algorithm's paper itself.

In [1]:
!git clone https://github.com/RapidsAtHKUST/SubgraphMatching.git

Cloning into 'SubgraphMatching'...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (278/278), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 278 (delta 9), reused 260 (delta 3), pack-reused 0
Receiving objects: 100% (278/278), 2.51 MiB | 12.30 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [2]:
from collections import defaultdict

In [3]:
class graph():
  def __init__(self, graphid, node2label, node2degree, edges):
    self.graphid = graphid
    self.node2label = node2label
    self.node2degree = node2degree
    self.edges = edges
    self.candidateset = defaultdict(set)
    self.label2node = defaultdict(set)
    for node in self.node2label:
      self.label2node[self.node2label[node]].add(node)
    self.phi = []
    self.phiparent = {}

  def reset(self):
    self.candidateset = defaultdict(set)
    self.phi = []
    self.phiparent = {}

In [4]:
def get_graph(filepath, filename):
  global qcount
  global gcount

  node2label = {}
  node2degree = {}
  edges = defaultdict(set)
  f = open(filepath, "r", encoding="utf-8")

  _, nodenum, edgenum = f.readline().strip().split()
  for i in range(int(nodenum)):
    _, nodeid, nodelabel, nodedegree = f.readline().strip().split()
    node2label[int(nodeid)] = int(nodelabel)
    node2degree[int(nodeid)] = int(nodedegree)  
  for i in range(int(edgenum)):
    _, node1, node2 = f.readline().strip().split()
    edges[int(node1)].add(int(node2))
    edges[int(node2)].add(int(node1))

  f.close()
  g = graph(filename, node2label, node2degree, edges)

  return g

In [5]:
qcount = 0
gcount = 0

import os
qs = []
qdir = "SubgraphMatching/test/query_graph"
for f in os.listdir(qdir):
  filepath = os.path.join(qdir, f)
  qs.append(get_graph(filepath, f))

gs = []
gdir = "SubgraphMatching/test/data_graph"
for f in os.listdir(gdir):
  filepath = os.path.join(gdir, f)
  gs.append(get_graph(filepath, f))

print(len(qs))
print(len(gs))

f = open("SubgraphMatching/test/expected_output.res", "r", encoding="utf-8")
lines = f.readlines()
f.close()

expects = {}
for line in lines:
  name, times = line.strip().split(":")
  expects[name + ".graph"] = int(times)
print(len(expects))

200
1
200


In [6]:
def graphQL(q, g, level):
  marks = set()
  for qnode in q.node2label:
    label = q.node2label[qnode]
    q.candidateset[qnode] |= g.label2node[label]
    for gnode in q.candidateset[qnode]:
      marks.add((qnode, gnode))
  
  for i in range(level):
    for qnode in q.node2label:
      for gnode in q.candidateset[qnode].copy():
        mark = (qnode, gnode)
        if mark not in marks:
          continue
        
        b = set()
        qneighbors = q.edges[qnode]
        gneighbors = g.edges[gnode]
        for qneighbor in qneighbors:
          for gneighbor in gneighbors:
            if gneighbor in q.candidateset[qneighbor]:
              b.add(qneighbor)
              break
        
        semiperfect = qneighbors - b
        if len(semiperfect) == 0:
          marks.remove(mark)
        else:
          q.candidateset[qnode].remove(gnode)
          for qneighbor in qneighbors:
            for gneighbor in gneighbors:
              if gneighbor in q.candidateset[qneighbor]:
                mark = (qneighbor, gneighbor)
                marks.add(mark)

    if len(marks) == 0:
      break

In [7]:
def RI(q):
  maxdegree = 0
  maxnode = 0
  visited = set()
  for node in q.node2degree:
    if maxdegree < q.node2degree[node]:
      maxdegree = q.node2degree[node]
      maxnode = node
  visited.add(maxnode)
  q.phi.append(maxnode)
  q.phiparent[maxnode] = -1
  while len(visited) != len(q.node2degree):
    m = len(q.phi)
    um = -1
    urank = (-1, -1, -1)
    for node in q.node2degree:
      if node in visited:
        continue
      v1 = visited & q.edges[node]
      v2 = set()
      for vis in visited:
        for vode in q.node2degree:
          if node == vode or vode in visited:
            continue
          if vode in q.edges[vis] and vode in q.edges[node]:
            v2.add(vis)
            break
      v3 = set()
      for vode in q.node2degree:
        flag = True
        if node == vode or vode in visited:
          continue            
        for vis in visited:
          if vode in q.edges[vis]:
            flag = False
            break
        if flag:
          v3.add(vode)
      rank = (len(v1), len(v2), len(v3))
      if urank <= rank:
        um = node
        urank = rank
    for parent in q.phi:
      if parent in q.edges[um]:
        q.phiparent[um] = parent
        break
    q.phi.append(um)
    visited.add(um)
    

In [10]:
def QuikSI(q, g, m, i, totalresult): # not equal to the original code
  if i == len(q.phi) + 1:
    totalresult.append(m.copy())
    return 
  result = {}
  u = -1
  for node in q.phi:
    if node not in m:
      u = node
      break

  lc = set()

  if i == 1:
    vs = g.label2node[q.node2label[u]]
    for v in vs:
      if g.node2degree[v] >= q.node2degree[u]:
        lc.add(v)
  else:
    for v in g.edges[m[q.phiparent[u]]]:
      if q.node2label[u] == g.node2label[v] and g.node2degree[v] >= q.node2degree[u]:
        flag = True
        for node in q.phi:
          if node == u:
            break
          if node == q.phiparent[u]:
            continue
          #if m[node] not in g.edges[v]:
          if v == m[node] or (m[node] not in g.edges[v] and (node in q.edges[u] or u in q.edges[node])):
            flag = False
            break
        if flag:
          lc.add(v)

  for node in lc:
    if node not in set(m.values()):
      m[u] = node
      QuikSI(q, g, m, i + 1, totalresult)
      m.pop(u)

In [11]:
queries = {}
for g in gs:
  for q in qs:
    q.reset()
    graphQL(q, g, 10)
    RI(q)
    totalresult = []
    QuikSI(q, g, {}, 1, totalresult)
    queries[q.graphid] = len(totalresult)

print(queries)

{'query_dense_16_170.graph': 18, 'query_dense_16_50.graph': 88, 'query_dense_16_72.graph': 24, 'query_dense_16_152.graph': 432, 'query_dense_16_86.graph': 1, 'query_dense_16_151.graph': 138, 'query_dense_16_95.graph': 354, 'query_dense_16_76.graph': 41, 'query_dense_16_168.graph': 75, 'query_dense_16_9.graph': 42, 'query_dense_16_181.graph': 8, 'query_dense_16_94.graph': 2, 'query_dense_16_88.graph': 12, 'query_dense_16_154.graph': 12, 'query_dense_16_37.graph': 1, 'query_dense_16_180.graph': 1, 'query_dense_16_164.graph': 480, 'query_dense_16_113.graph': 6, 'query_dense_16_188.graph': 8, 'query_dense_16_104.graph': 38, 'query_dense_16_42.graph': 32, 'query_dense_16_124.graph': 2, 'query_dense_16_174.graph': 6, 'query_dense_16_71.graph': 9, 'query_dense_16_87.graph': 12, 'query_dense_16_39.graph': 1, 'query_dense_16_65.graph': 2, 'query_dense_16_11.graph': 288, 'query_dense_16_14.graph': 2, 'query_dense_16_145.graph': 1, 'query_dense_16_30.graph': 2, 'query_dense_16_58.graph': 3, 'quer

In [12]:
flag = True
for name in expects:
  if expects[name] != queries[name]:
    print(name)
    flag = False
if flag:
  print("correct")

correct
